In [1]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
df = pd.read_csv('train_imputed.csv')


C:\Users\ferha\AppData\Local\Temp\ipykernel_3980\2319938952.py:6: DtypeWarning: Columns (12,27,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_imputed.csv')


In [2]:

# Get the list of column names
column_names = df.columns.tolist()

for i, column in enumerate(column_names):
    print(f"{i+1}. {column}")


1. Basvuru Yili
2. Degerlendirme Puani
3. Cinsiyet
4. Dogum Yeri
5. Ikametgah Sehri
6. Universite Adi
7. Universite Turu
8. Burslu ise Burs Yuzdesi
9. Burs Aliyor mu?
10. Bölüm
11. Universite Kacinci Sinif
12. Universite Not Ortalamasi
13. Daha Once Baska Bir Universiteden Mezun Olmus
14. Lise Turu
15. Lise Mezuniyet Notu
16. Baska Bir Kurumdan Burs Aliyor mu?
17. Burs Aldigi Baska Kurum
18. Anne Egitim Durumu
19. Anne Calisma Durumu
20. Anne Sektor
21. Baba Egitim Durumu
22. Baba Calisma Durumu
23. Baba Sektor
24. Kardes Sayisi
25. Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?
26. Profesyonel Bir Spor Daliyla Mesgul musunuz?
27. Aktif olarak bir STK üyesi misiniz?
28. Stk Projesine Katildiniz Mi?
29. Girisimcilikle Ilgili Deneyiminiz Var Mi?
30. Ingilizce Biliyor musunuz?
31. Ingilizce Seviyeniz?
32. Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite
33. id
34. Yas
35. lise giris ort


In [3]:
categorical_features = ['Basvuru Yili', 'Cinsiyet', 'Dogum Yeri', 'Ikametgah Sehri',
                        'Universite Adi', 'Universite Turu', 'Burs Aliyor mu?','Bölüm','Daha Once Baska Bir Universiteden Mezun Olmus',
                        'Lise Turu','Baska Bir Kurumdan Burs Aliyor mu?','Anne Calisma Durumu','Anne Sektor','Baba Calisma Durumu',
                        'Baba Sektor','Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?','Profesyonel Bir Spor Daliyla Mesgul musunuz?','Aktif olarak bir STK üyesi misiniz?',
                        'Stk Projesine Katildiniz Mi?','Girisimcilikle Ilgili Deneyiminiz Var Mi?',
                        'Ingilizce Biliyor musunuz?', 'Burs Aldigi Baska Kurum', 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']

numerical_features = [column for column in df.columns if column not in categorical_features]
print(numerical_features)

# Fill missing values in categorical features
df[categorical_features] = df[categorical_features].fillna("0").astype(str)

# Fill missing values in numerical features
df[numerical_features] = df[numerical_features].fillna(0)


X = df.drop(['Degerlendirme Puani', 'id'], axis=1)  # Drop target and id columns
y = df['Degerlendirme Puani']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.fillna(y_train.mean())
y_val = y_val.fillna(y_val.mean())


['Degerlendirme Puani', 'Burslu ise Burs Yuzdesi', 'Universite Kacinci Sinif', 'Universite Not Ortalamasi', 'Lise Mezuniyet Notu', 'Anne Egitim Durumu', 'Baba Egitim Durumu', 'Kardes Sayisi', 'Ingilizce Seviyeniz?', 'id', 'Yas', 'lise giris ort']


In [6]:
categorical_features


['Basvuru Yili',
 'Cinsiyet',
 'Dogum Yeri',
 'Ikametgah Sehri',
 'Universite Adi',
 'Universite Turu',
 'Burs Aliyor mu?',
 'Bölüm',
 'Daha Once Baska Bir Universiteden Mezun Olmus',
 'Lise Turu',
 'Baska Bir Kurumdan Burs Aliyor mu?',
 'Anne Calisma Durumu',
 'Anne Sektor',
 'Baba Calisma Durumu',
 'Baba Sektor',
 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
 'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
 'Aktif olarak bir STK üyesi misiniz?',
 'Stk Projesine Katildiniz Mi?',
 'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
 'Ingilizce Biliyor musunuz?',
 'Burs Aldigi Baska Kurum',
 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']

In [5]:

numerical_features

['Degerlendirme Puani',
 'Burslu ise Burs Yuzdesi',
 'Universite Kacinci Sinif',
 'Universite Not Ortalamasi',
 'Lise Mezuniyet Notu',
 'Anne Egitim Durumu',
 'Baba Egitim Durumu',
 'Kardes Sayisi',
 'Ingilizce Seviyeniz?',
 'id',
 'Yas',
 'lise giris ort']

In [10]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=7,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    use_best_model=True,      # Use the best model from the iterations
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)



# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_1.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.6793832	test: 16.7243154	best: 16.7243154 (0)	total: 51.4ms	remaining: 2m 34s
100:	learn: 5.7381406	test: 5.7697306	best: 5.7697306 (100)	total: 5.53s	remaining: 2m 38s
200:	learn: 5.5611116	test: 5.6919476	best: 5.6919476 (200)	total: 11.1s	remaining: 2m 34s
300:	learn: 5.4502855	test: 5.6566386	best: 5.6561427 (296)	total: 16.6s	remaining: 2m 28s
400:	learn: 5.3614487	test: 5.6404364	best: 5.6399791 (394)	total: 22.3s	remaining: 2m 24s
500:	learn: 5.2866732	test: 5.6351745	best: 5.6351745 (500)	total: 27.8s	remaining: 2m 18s
600:	learn: 5.2033488	test: 5.6290471	best: 5.6290471 (600)	total: 33.4s	remaining: 2m 13s
700:	learn: 5.1191565	test: 5.6261742	best: 5.6259002 (699)	total: 39s	remaining: 2m 7s
800:	learn: 5.0414263	test: 5.6255008	best: 5.6245162 (757)	total: 44.6s	remaining: 2m 2s
900:	learn: 4.9683816	test: 5.6267303	best: 5.6245162 (757)	total: 50.2s	remaining: 1m 57s
1000:	learn: 4.8932672	test: 5.6282879	best: 5.6245162 (757)	total: 55.9s	remaining: 1m 51s
11

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=8,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    use_best_model=True,      # Use the best model from the iterations
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_2_depth8.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.6618244	test: 16.7044473	best: 16.7044473 (0)	total: 55.9ms	remaining: 2m 47s
100:	learn: 5.6519965	test: 5.7593207	best: 5.7593207 (100)	total: 6.68s	remaining: 3m 11s
200:	learn: 5.4511759	test: 5.6867060	best: 5.6865075 (197)	total: 13.1s	remaining: 3m 2s
300:	learn: 5.3006321	test: 5.6534167	best: 5.6534167 (300)	total: 19.3s	remaining: 2m 52s
400:	learn: 5.1617291	test: 5.6479410	best: 5.6476277 (397)	total: 25.7s	remaining: 2m 46s
500:	learn: 5.0445965	test: 5.6453108	best: 5.6449594 (491)	total: 32.1s	remaining: 2m 40s
600:	learn: 4.9141990	test: 5.6411101	best: 5.6407930 (599)	total: 38.7s	remaining: 2m 34s
700:	learn: 4.7911977	test: 5.6391938	best: 5.6380653 (641)	total: 45s	remaining: 2m 27s
800:	learn: 4.6776196	test: 5.6358036	best: 5.6354838 (796)	total: 51.4s	remaining: 2m 21s
900:	learn: 4.5740304	test: 5.6392055	best: 5.6352922 (807)	total: 58.3s	remaining: 2m 15s
1000:	learn: 4.4718542	test: 5.6407760	best: 5.6352922 (807)	total: 1m 5s	remaining: 2m 9s
11

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [12]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=6,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    use_best_model=True,      # Use the best model from the iterations
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_3_depth6.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.7220566	test: 16.7731494	best: 16.7731494 (0)	total: 49.8ms	remaining: 2m 29s
100:	learn: 5.8311021	test: 5.8238628	best: 5.8238628 (100)	total: 4.88s	remaining: 2m 20s
200:	learn: 5.6574482	test: 5.7085092	best: 5.7085092 (200)	total: 9.85s	remaining: 2m 17s
300:	learn: 5.5648045	test: 5.6756182	best: 5.6756182 (300)	total: 14.8s	remaining: 2m 12s
400:	learn: 5.5000702	test: 5.6553582	best: 5.6553582 (400)	total: 19.7s	remaining: 2m 7s
500:	learn: 5.4404848	test: 5.6459183	best: 5.6458467 (499)	total: 24.7s	remaining: 2m 3s
600:	learn: 5.3828469	test: 5.6339158	best: 5.6339158 (600)	total: 29.5s	remaining: 1m 57s
700:	learn: 5.3308285	test: 5.6286752	best: 5.6286752 (700)	total: 34.3s	remaining: 1m 52s
800:	learn: 5.2799813	test: 5.6199238	best: 5.6194608 (793)	total: 39.3s	remaining: 1m 47s
900:	learn: 5.2249941	test: 5.6162618	best: 5.6150995 (886)	total: 44.2s	remaining: 1m 42s
1000:	learn: 5.1787515	test: 5.6116144	best: 5.6109110 (990)	total: 49.3s	remaining: 1m 38s


c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=4000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=5,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_depth5.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.7847517	test: 16.8340363	best: 16.8340363 (0)	total: 42.1ms	remaining: 2m 48s
100:	learn: 5.9225708	test: 5.8979998	best: 5.8979998 (100)	total: 4.08s	remaining: 2m 37s
200:	learn: 5.7435376	test: 5.7669168	best: 5.7669168 (200)	total: 8.06s	remaining: 2m 32s
300:	learn: 5.6570027	test: 5.7160201	best: 5.7159517 (299)	total: 12.1s	remaining: 2m 28s
400:	learn: 5.6003693	test: 5.6885969	best: 5.6885969 (400)	total: 16.1s	remaining: 2m 24s
500:	learn: 5.5609108	test: 5.6789835	best: 5.6789214 (483)	total: 20.3s	remaining: 2m 21s
600:	learn: 5.5176336	test: 5.6651846	best: 5.6650831 (595)	total: 24.4s	remaining: 2m 18s
700:	learn: 5.4839545	test: 5.6543513	best: 5.6542476 (699)	total: 28.6s	remaining: 2m 14s
800:	learn: 5.4436200	test: 5.6441241	best: 5.6441241 (800)	total: 32.7s	remaining: 2m 10s
900:	learn: 5.4125164	test: 5.6388419	best: 5.6378828 (888)	total: 36.9s	remaining: 2m 7s
1000:	learn: 5.3836469	test: 5.6375257	best: 5.6370625 (914)	total: 41.1s	remaining: 2m 3s


c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE: 5.609912655804014


In [16]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=20000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=3,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    use_best_model=True,      # Use the best model from the iterations
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_depth3.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.9809586	test: 17.0311727	best: 17.0311727 (0)	total: 26.7ms	remaining: 8m 54s
100:	learn: 6.2369045	test: 6.1795902	best: 6.1795902 (100)	total: 2.47s	remaining: 8m 6s
200:	learn: 5.9407204	test: 5.8995846	best: 5.8995846 (200)	total: 4.79s	remaining: 7m 51s
300:	learn: 5.8421464	test: 5.8211316	best: 5.8211316 (300)	total: 7.24s	remaining: 7m 53s
400:	learn: 5.7881013	test: 5.7809349	best: 5.7807979 (398)	total: 9.58s	remaining: 7m 48s
500:	learn: 5.7477766	test: 5.7552056	best: 5.7552056 (500)	total: 11.9s	remaining: 7m 43s
600:	learn: 5.7170591	test: 5.7371608	best: 5.7371608 (600)	total: 14.4s	remaining: 7m 43s
700:	learn: 5.6913807	test: 5.7233952	best: 5.7233952 (700)	total: 16.8s	remaining: 7m 41s
800:	learn: 5.6725665	test: 5.7135075	best: 5.7134661 (798)	total: 19.2s	remaining: 7m 39s
900:	learn: 5.6578280	test: 5.7084809	best: 5.7083569 (898)	total: 21.6s	remaining: 7m 37s
1000:	learn: 5.6430396	test: 5.7029518	best: 5.7028668 (997)	total: 24.1s	remaining: 7m 36s

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE: 5.614966370620446


In [17]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=10000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=6,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    use_best_model=True,      # Use the best model from the iterations
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_big_depth6.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.7220566	test: 16.7731494	best: 16.7731494 (0)	total: 45.4ms	remaining: 7m 33s
100:	learn: 5.8342431	test: 5.8288825	best: 5.8288825 (100)	total: 5.32s	remaining: 8m 41s
200:	learn: 5.6557654	test: 5.7114331	best: 5.7114331 (200)	total: 10.6s	remaining: 8m 39s
300:	learn: 5.5616599	test: 5.6724720	best: 5.6724720 (300)	total: 15.6s	remaining: 8m 24s
400:	learn: 5.4995450	test: 5.6541190	best: 5.6541190 (400)	total: 20.7s	remaining: 8m 16s
500:	learn: 5.4385961	test: 5.6401171	best: 5.6401171 (500)	total: 25.9s	remaining: 8m 11s
600:	learn: 5.3832596	test: 5.6300245	best: 5.6300215 (599)	total: 30.9s	remaining: 8m 2s
700:	learn: 5.3314219	test: 5.6266473	best: 5.6264569 (679)	total: 36.1s	remaining: 7m 59s
800:	learn: 5.2807913	test: 5.6214663	best: 5.6210527 (793)	total: 41.3s	remaining: 7m 54s
900:	learn: 5.2320289	test: 5.6182819	best: 5.6168069 (884)	total: 46.5s	remaining: 7m 49s
1000:	learn: 5.1814270	test: 5.6101028	best: 5.6098537 (999)	total: 52.4s	remaining: 7m 51s

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
